# Chapter06

In [2]:
!wget -nc http://www.cl.ecei.tohoku.ac.jp/nlp100/data/nlp.txt -P data

--2017-05-16 03:56:39--  http://www.cl.ecei.tohoku.ac.jp/nlp100/data/nlp.txt
Resolving www.cl.ecei.tohoku.ac.jp... 130.34.192.83
Connecting to www.cl.ecei.tohoku.ac.jp|130.34.192.83|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8594 (8.4K) [text/plain]
Saving to: ‘data/nlp.txt’

nlp.txt             100%[===================>]   8.39K  --.-KB/s    in 0s      

2017-05-16 03:56:39 (31.9 MB/s) - ‘data/nlp.txt’ saved [8594/8594]



In [1]:
!head -n 4 data/nlp.txt

Natural language processing
From Wikipedia, the free encyclopedia

Natural language processing (NLP) is a field of computer science, artificial intelligence, and linguistics concerned with the interactions between computers and human (natural) languages. As such, NLP is related to the area of humani-computer interaction. Many challenges in NLP involve natural language understanding, that is, enabling computers to derive meaning from human or natural language input, and others involve natural language generation.


# 50. 文区切り
(. or ; or : or ? or !) → 空白文字 → 英大文字というパターンを文の区切りと見なし，入力された文書を1行1文の形式で出力せよ．

In [27]:
%%file q50.sh
pcregrep -o ".+?[.;:?!]\s(?=[A-Z])" data/nlp.txt | head

Overwriting q50.sh


In [28]:
!chmod u+x q50.sh
!./q50.sh

Natural language processing (NLP) is a field of computer science, artificial intelligence, and linguistics concerned with the interactions between computers and human (natural) languages. 
As such, NLP is related to the area of humani-computer interaction. 
The history of NLP generally starts in the 1950s, although work can be found from earlier periods. 
The Georgetown experiment in 1954 involved fully automatic translation of more than sixty Russian sentences into English. 
The authors claimed that within three or five years, machine translation would be a solved problem. 
However, real progress was much slower, and after the ALPAC report in 1966, which found that ten year long research had failed to fulfill the expectations, funding for machine translation was dramatically reduced. 
Some notably successful NLP systems developed in the 1960s were SHRDLU, a natural language system working in restricted "blocks worlds" with restricted vocabularies, and ELIZA, a simulation of a Rogerian

# 51. 単語の切り出し
空白を単語の区切りとみなし，50の出力を入力として受け取り，1行1単語の形式で出力せよ．ただし，文の終端では空行を出力せよ

In [29]:
%%file q51.sh
pcregrep -o ".+?[.;:?!]\s(?=[A-Z])" data/nlp.txt |
pcregrep -o "[a-zA-Z]+?[\s\.]" | tr "." "\n" | tee work/nlp_parsed.txt | head -n30

Writing q51.sh


In [30]:
!chmod u+x q51.sh
!./q51.sh

Natural 
language 
processing 
is 
a 
field 
of 
computer 
artificial 
and 
linguistics 
concerned 
with 
the 
interactions 
between 
computers 
and 
human 
languages

As 
NLP 
is 
related 
to 
the 
area 
of 
computer 


# 52. ステミング
51の出力を入力として受け取り，Porterのステミングアルゴリズムを適用し，単語と語幹をタブ区切り形式で出力せよ． Pythonでは，Porterのステミングアルゴリズムの実装としてstemmingモジュールを利用するとよい．

In [59]:
!pip install nltk

    100% |████████████████████████████████| 1.2MB 856kB/s ta 0:00:011
  Running setup.py install for nltk ... - \ | / - \ | / - \ | / - \ | done


In [31]:
%%file q52.py
from nltk.stem.porter import PorterStemmer
with open("work/nlp_parsed.txt", "r") as f1, open("work/nlp_stemmed.txt", "w") as f2:
    stemmer = PorterStemmer()
    stemmed = [stemmer.stem(word.strip()) for word in f1]
    f2.write("\n".join(stemmed))

Writing q52.py


In [4]:
!head work/nlp_stemmed.txt

natur
languag
process
is
a
field
of
comput
artifici
and


# 53. Tokenization
Stanford Core NLPを用い，入力テキストの解析結果をXML形式で得よ．また，このXMLファイルを読み込み，入力テキストを1行1単語の形式で出力せよ．

In [5]:
! java -cp "stanford-corenlp-full-2017-06-09/*" -Xmx2g edu.stanford.nlp.pipeline.StanfordCoreNLP -annotators tokenize,ssplit,pos,lemma,ner,parse,dcoref -file data/nlp.txt -outputDirectory ./work

[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator pos
[main] INFO edu.stanford.nlp.tagger.maxent.MaxentTagger - Loading POS tagger from edu/stanford/nlp/models/pos-tagger/english-left3words/english-left3words-distsim.tagger ... done [1.7 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
[main] INFO edu.stanford.nlp.ie.AbstractSequenceClassifier - Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [4.3 sec].
[main] INFO edu.stanford.nlp.ie.AbstractSequenceClassifier - Loading classifier from edu/stanford/nlp/models/ner/english.muc.7cla

In [7]:
!head -n30 work/nlp.txt.xml

In [32]:
%%file q53.py
import xml.etree.ElementTree as ET
tree = ET.parse('work/nlp.txt.xml')
root = tree.getroot()
for word in root.iter('word'):
    print(word.text)

Writing q53.py


# 54. 品詞タグ付け
Stanford Core NLPの解析結果XMLを読み込み，単語，レンマ，品詞をタブ区切り形式で出力せよ．

In [33]:
%%file q54.py
import xml.etree.ElementTree as ET
tree = ET.parse('work/nlp.txt.xml')
root = tree.getroot()
for token in root.iter('token'):
    print("{}\t{}\t{}".format(token[0].text, token[1].text, token[4].text))

Writing q54.py


# 55. 固有表現抽出
入力文中の人名をすべて抜き出せ．

In [34]:
%%file q55.py
import xml.etree.ElementTree as ET
tree = ET.parse('work/nlp.txt.xml')
root = tree.getroot()
for token in root.findall(".//*[NER='PERSON']"): 
    print(token[0].text)

Writing q55.py


# 56. 共参照解析
Stanford Core NLPの共参照解析の結果に基づき，文中の参照表現（mention）を代表参照表現（representative mention）に置換せよ．ただし，置換するときは，「代表参照表現（参照表現）」のように，元の参照表現が分かるように配慮せよ．

In [35]:
%%file q56.py
import xml.etree.ElementTree as ET
tree = ET.parse('work/nlp.txt.xml')
root = tree.getroot()
for coref in root.findall('.//coreference/coreference'):
    rep_mention = coref.find("*[@representative='true']")
    for mention in coref:
        word_start, word_end = mention[-1].text.split()[0], mention[-1].text.split()[-1]
        i_sent, i_start, i_end = mention[0].text, mention[1].text, str(int(mention[2].text)-1)
        root.find(".//sentence[@id='{}']/tokens/token[@id='{}']".format(i_sent, i_start))[0].text \
        = " [" + rep_mention[-1].text + "](" + word_start
        root.find(".//sentence[@id='{}']/tokens/token[@id='{}']".format(i_sent, i_end))[0].text \
        = word_end + ")"
tree.write('work/nlp_coreference.txt.xml')

#出力の整形
with open('work/nlp_coreferene.txt', 'w') as f:
    tree = ET.parse('work/nlp_coreference.txt.xml')
    root = tree.getroot()
    for sentence in root.iter('sentence'):
        for word in sentence.iter('word'):
            f.write(word.text + " ")
        f.write("\n")
! gsed -E "s/\s(\W)/\1/g" work/nlp_coreferene.txt

Writing q56.py


In [12]:
#出力の整形
with open('work/nlp_coreferene.txt', 'w') as f:
    tree = ET.parse('work/nlp_coreference.txt.xml')
    root = tree.getroot()
    for sentence in root.iter('sentence'):
        for word in sentence.iter('word'):
            f.write(word.text + " ")
        f.write("\n")
! gsed -E "s/\s(\W)/\1/g" work/nlp_coreferene.txt

Natural language processing From Wikipedia, [the free encyclopedia Natural language processing-LRB- NLP-RRB-](the free encyclopedia Natural language processing-LRB- NLP-RRB-) is [the free encyclopedia Natural language processing-LRB- NLP-RRB-](a field of computer science), artificial intelligence, and linguistics concerned with the interactions between computers) and human-LRB- natural-RRB- languages. 
As such, NLP is related to the area of humani-computer interaction. 
Many challenges in NLP involve natural language understanding, that is, enabling computers) to derive meaning from human or natural language input, and others involve natural language generation. 
History The history of NLP) generally starts in the 1950s, although work can be found from earlier periods. 
In 1950, [Alan Turing](Alan Turing) published an article titled`` Computing Machinery and Intelligence'' which proposed what is now called the Turing) test as a criterion of intelligence. 
The Georgetown experiment in 1

# 57. 係り受け解析
Stanford Core NLPの係り受け解析の結果（collapsed-dependencies）を有向グラフとして可視化せよ．可視化には，係り受け木をDOT言語に変換し，Graphvizを用いるとよい．また，Pythonから有向グラフを直接的に可視化するには，pydotを使うとよい．

In [36]:
%%file q57.py
from graphviz import Digraph
import xml.etree.ElementTree as ET
tree = ET.parse('work/nlp.txt.xml')
root = tree.getroot()

def dependency_tree(i_sent):
    G = Digraph(format='png')
    G.attr('node', shape='circle')
    G.node("0", "ROOT")
    root = tree.getroot()
    for dep in root.findall(".//sentence[@id='{}']/dependencies[@type='collapsed-dependencies']/dep".format(i_sent)):
        G.node(dep[1].get('idx'), dep[1].text)
        G.edge(dep[0].get('idx'), dep[1].get('idx'))
    G.render("work/57")
    print(G)

dependency_tree(2)

Writing q57.py


In [37]:
!python q57.py

digraph {
	node [shape=circle]
	0 [label=ROOT]
	6 [label=related]
		0 -> 6
	1 [label=As]
		2 -> 1
	2 [label=such]
		6 -> 2
	3 [label=","]
		6 -> 3
	4 [label=NLP]
		6 -> 4
	5 [label=is]
		6 -> 5
	7 [label=to]
		9 -> 7
	8 [label=the]
		9 -> 8
	9 [label=area]
		6 -> 9
	10 [label=of]
		12 -> 10
	11 [label="humani-computer"]
		12 -> 11
	12 [label=interaction]
		9 -> 12
	13 [label="."]
		6 -> 13
}


<img src="work/57.png">

# 58. タプルの抽出
Stanford Core NLPの係り受け解析の結果（collapsed-dependencies）に基づき，「主語 述語 目的語」の組をタブ区切り形式で出力せよ．ただし，主語，述語，目的語の定義は以下を参考にせよ．

述語: nsubj関係とdobj関係の子（dependant）を持つ単語
主語: 述語からnsubj関係にある子（dependent）
目的語: 述語からdobj関係にある子（dependent）

In [42]:
%%file q58.py
import xml.etree.ElementTree as ET
def dep_nd():
    tree = ET.parse('work/nlp.txt.xml')
    root = tree.getroot()
    dep_n = [(dep_nsubj[0], dep_nsubj[1]) 
     for dep_nsubj in root.findall(".//dependencies[@type='collapsed-dependencies']/dep[@type='nsubj']")]
    dep_d = [(dep_nsubj[0], dep_nsubj[1]) 
     for dep_nsubj in root.findall(".//dependencies[@type='collapsed-dependencies']/dep[@type='dobj']")]
    make_tuple(dep_n, dep_d)

def make_tuple(dep_n, dep_d):
    for n in dep_n:
        for s in dep_d:
            if n[0].get('id') == s[0].get('id'):
                print(n[1].text, n[0].text, s[1].text)

dep_nd()   

Writing q58.py


In [43]:
!python q58.py

processing field computers
processing field meaning
processing field generation
processing field article
processing field test
processing field translation
processing field expectations
processing field information
processing field interaction
processing field base
processing field response
processing field conceptual
processing field information
processing field sort
processing field approach
processing field algorithms
processing field systems
processing field use
processing field decisions
processing field weights
processing field data
processing field which
processing field input
processing field errors
processing field results
processing field subtasks
processing field advantage
processing field combination
processing field results
processing field content
processing field algorithms
processing field coding
processing field algorithms
processing field rules
processing field set
processing field systems
processing field decisions
processing field weights
processing field advantage


# 59. S式の解析
Stanford Core NLPの句構造解析の結果（S式）を読み込み，文中のすべての名詞句（NP）を表示せよ．入れ子になっている名詞句もすべて表示すること.

S式のパーサー(ネットで見つけた)を利用
http://www.unixuser.org/~euske/python/index-j.html  

In [84]:
%%file q59.py
import sexpr
def parse():
    tree = ET.parse('work/nlp.txt.xml')
    root = tree.getroot()
    for s in root.findall(".//parse"):
        list_s = sexpr.str2sexpr(s.text)
        search_np(list_s)

def search_np(list_s):
    if len(list_s) == 1:
        search_np(list_s[0])
    elif list_s[0] == "NP":
        print_np(list_s)
        for element in list_s[1:]:
            search_np(element)
        print()
    elif isinstance(list_s[1], list):
        for element in list_s[1:]:
            search_np(element)
    return

def print_np(list_s):
    for element_np in list_s[1:]:
        if isinstance(element_np[1], str):
            print(element_np[1], end=" ")
        else:
            print_np(element_np)

parse()

Writing q59.py
